# Use Spark for R to load data and run SQL queries
This notebook introduces basic Spark concepts and helps you to start using Spark for R.

Some familiarity with R is recommended.

In this notebook, you'll use the publicly available **mtcars** data set from *Motor Trend* magazine to learn some basic R. You'll learn how to load data, create a Spark DataFrame, aggregate data, run mathematical formulas, and run SQL queries against the data.

## Table of contents
This notebook contains these main sections:

1. [Load a DataFrame](#Load_a_DataFrame)
2. [Initialize an SQLContext](#Initialize_an_SQLContext)
3. [Create a Spark DataFrame](#Create_a_Spark_DataFrame)
4. [Aggregate data after grouping by columns](#Aggregate_data_after_grouping_by_columns)
5. [Operate on columns](#Operate_on_columns)
6. [Run SQL queries from the Spark DataFrame](#Run_SQL_queries_from_the_Spark_DataFrame)

<a id='Load_a_DataFrame'></a>
## 1. Load a DataFrame
A DataFrame is a distributed collection of data that is organized into named columns. The built-in R DataFrame called **mtcars** includes observations on the following 11 variables:

`[, 1]	mpg     Miles / (US) gallon`  
`[, 2]	cyl     Number of cylinders`  
`[, 3]	disp	Displacement (cu. in.)`  
`[, 4]	hp      Gross horsepower`  
`[, 5]	drat    Rear axle ratio`  
`[, 6]	wt      Weight (1000 lbs)`  
`[, 7]	qsec    1/4 mile time (seconds)`  
`[, 8]	vs      0 = V-engine, 1 = straight engine`  
`[, 9]	am      Transmission (0 = automatic, 1 = manual)`  
`[,10]	gear    Number of forward gears`  
`[,11]	carb    Number of carburetors`

Preview the first 3 rows of the DataFrame by using the head() function:

In [ ]:
head(mtcars, 3)

Convert the car name data, which appears in the row names, into an actual column so that Spark can read it as a column:

In [ ]:
mtcars$car <- rownames(mtcars)
mtcars <- mtcars[,c(12,1:11)]
rownames(mtcars) <- 1:nrow(mtcars)
head(mtcars)

<a id='Initialize_an_SQLContext'></a>
## 2. Initialize an SQLContext
To work with a DataFrame, you need an SQLContext. You create this SQLContext by using `sparkRSQL.init(sc)`. A SparkContext named sc, which has been created for you, is used to initialize the SQLContext:

In [ ]:
sqlContext <- sparkRSQL.init(sc)

<a id='Create_a_Spark_DataFrame'></a>
## 3. Create a Spark DataFrame
Using the SQLContext and the loaded local DataFrame, create a Spark DataFrame and print the schema, or structure, of the DataFrame:

In [ ]:
sdf <- createDataFrame(sqlContext, mtcars) 
printSchema(sdf)

Display the content of the Spark DataFrame:

In [ ]:
SparkR::head(sdf, 32)

Try different ways of retrieving subsets of the data. For example, get the first 5 values in the **mpg** column:

In [ ]:
SparkR::head(select(sdf, sdf$mpg),5)

Filter the DataFrame to retain only rows with **mpg** values that are less than 18:

In [ ]:
SparkR::head(SparkR::filter(sdf, sdf$mpg < 18))

<a id='Aggregate_data_after_grouping_by_columns'></a>
## 4. Aggregate data after grouping by columns
Spark DataFrames support a number of common functions to aggregate data after grouping. For example, you can compute the average weight of cars as a function of the number of cylinders:

In [ ]:
SparkR::head(summarize(groupBy(sdf, sdf$cyl), wtavg = avg(sdf$wt)))

You can also sort the output from the aggregation to determine the most popular cylinder configuration in the DataFrame:

In [ ]:
car_counts <-summarize(groupBy(sdf, sdf$cyl), count = n(sdf$wt))
SparkR::head(arrange(car_counts, desc(car_counts$count)))

<a id='Operate_on_columns'></a>
## 5. Operate on columns
SparkR provides a number of functions that you can apply directly to columns for data processing. In the following example, a basic arithmetic function converts lbs to metric tons:

In [ ]:
sdf$wtTon <- sdf$wt * 0.45
SparkR::head(select(sdf, sdf$car, sdf$wt, sdf$wtTon),6)

<a id='Run_SQL_queries_from_the_Spark_DataFrame'></a>
## 6. Run SQL queries from the Spark DataFrame
You can register a Spark DataFrame as a temporary table and then run SQL queries over the data. The `sql` function enables an application to run SQL queries programmatically and returns the result as a DataFrame:

In [ ]:
registerTempTable(sdf, "cars")

highgearcars <- sql(sqlContext, "SELECT car, gear FROM cars WHERE gear >= 5")
SparkR::head(highgearcars)

## That's it!
You successfully completed this notebook! You learned how to load a DataFrame, view and filter the data, aggregate the data, perform operations on the data in specific columns, and run SQL queries against the data. For more information about Spark, see the [Spark Quick Start Guide](http://spark.apache.org/docs/latest/quick-start.html).

## Want to learn more?
### Free courses on <a href="https://bigdatauniversity.com/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu/" rel="noopener noreferrer" target="_blank">Big Data University</a>: <a href="https://bigdatauniversity.com/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu" rel="noopener noreferrer" target="_blank"><img src = "https://ibm.box.com/shared/static/xomeu7dacwufkoawbg3owc8wzuezltn6.png" width=600px> </a>

### Authors

**Saeed Aghabozorgi**, PhD, is a Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge. He is a researcher in the data mining field and an expert in developing advanced analytic methods like machine learning and statistical modelling on large data sets.

**Polong Lin** is a Data Scientist at IBM in Canada. Under the Emerging Technologies division, Polong is responsible for educating the next generation of data scientists through Big Data University. Polong is a regular speaker in conferences and meetups, and holds an M.Sc. in Cognitive Psychology.

Copyright © 2016 Big Data University. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/" rel="noopener noreferrer" target="_blank">MIT License</a>.